Imports


In [108]:
import pandas as pd

Load data

In [186]:
df1 = pd.read_excel("metaClean43Brightspace.xlsx")
df2 = pd.read_excel("sales.xlsx")

Remove duplicates and merge data

In [169]:
# Remove duplicates from df1 and df2
df1 = df1.drop_duplicates(subset='title')
df2 = df2.drop_duplicates(subset='title')

# Merge the DataFrames
merged_df = df1.merge(df2, on='title', how='inner')

<bound method NDFrame.describe of 0       10 Cloverfield Lane
1                  10 Years
2          100 Bloody Acres
3               100 Streets
4                1001 Grams
               ...         
8120              Zookeeper
8121              Zoolander
8122            Zoolander 2
8123                   Zoom
8124               Zootopia
Name: title, Length: 8125, dtype: object>

Keep needed features

In [170]:
columns_to_keep = ['title', 'genre_x', 'genre_y']
df = merged_df[columns_to_keep]

Split genres in different columns

In [171]:
# Split the 'genres' column by comma and create new columns for each genre
split_genres = df['genre_x'].str.split(',', expand=True)
df['genre2'] = split_genres[0]
df['genre3'] = split_genres[1]
df['genre4'] = split_genres[2]
df['genre5'] = split_genres[3]
df['genre6'] = split_genres[4]
df['genre7'] = split_genres[5]
df['genre8'] = split_genres[6]
df['genre9'] = split_genres[7]
df['genre10'] = split_genres[8]
df['genre11'] = split_genres[9]

C:\Users\daanv\AppData\Local\Temp\ipykernel_10120\3941831681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre2'] = split_genres[0]
C:\Users\daanv\AppData\Local\Temp\ipykernel_10120\3941831681.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre3'] = split_genres[1]
C:\Users\daanv\AppData\Local\Temp\ipykernel_10120\3941831681.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

(8125, 13)

Drop exploded column and rename column y

In [172]:
df = df.drop(columns=['genre_x'])
df = df.rename(columns={'genre_y': 'genre1'})

Check and delete duplicate genres

In [173]:
columns_to_check = ['genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10']

for index, row in df.iterrows():
    genre1_value = row['genre1']
    for col in columns_to_check:
        if row[col] == genre1_value:
            df.at[index, col] = None 

(8125, 12)

Check genre columns for unique columns, create a new dataframe including a column of unique genres and give them a primary key

In [175]:
# Concatenate the 'genre' columns into a single Series
genre_series = df[['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10', 'genre11']].stack().reset_index(drop=True)

# Get unique genre values and assign IDs
unique_genres = genre_series.unique()
genre_to_id = {genre: id for id, genre in enumerate(unique_genres)}

# Create a DataFrame with genre IDs
genre_id_df = pd.DataFrame({'Genre': unique_genres, 'Genre_ID': [genre_to_id[genre] for genre in unique_genres]})

genre_id_df['Genre_ID'] = genre_id_df['Genre_ID'] + 1

genre_id_df = genre_id_df[['Genre_ID', 'Genre']]


Save file

In [177]:
genre_id_df.to_csv('genre.csv')

Take the datafame with the genres seperated in columns and combine the genre columns into one column

In [178]:
# List of genre column names
df = [f'genre{i}' for i in range(1, 12)]

# Combine genre columns into a single column as a list
df['Combined_Genres'] = df.apply(lambda row: [row[col] for col in genre_columns if pd.notna(row[col])], axis=1)

# Drop the individual genre columns
df.drop(columns=genre_columns, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

Explode the genres in multiple rows

In [179]:
# Explode the 'Combined_Genres' column to create separate rows
dfnew = genre_id_df.explode('Combined_Genres', ignore_index=True)

# Reset the index
dfnew.reset_index(drop=True, inplace=True)

dfnew = dfnew.rename(columns={'Combined_Genres': 'Genre'})

Merge the id's of the list of unique genres to the genres per row per movie

In [181]:
# Perform a left merge to replace genre with genre_ID
merged_df = dfnew.merge(genre_id_df, on='Genre', how='left')

# Drop the original 'Genre' column and rename 'Genre_ID' to 'Genre'
merged_df.drop(columns='Genre', inplace=True)
merged_df.rename(columns={'Genre_ID': 'Genre'}, inplace=True)
if 'Unnamed: 0_x' in merged_df.columns:
    merged_df.drop(columns='Unnamed: 0_x', inplace=True)

if 'Unnamed: 0_y' in merged_df.columns:
    merged_df.drop(columns='Unnamed: 0_y', inplace=True)

Add primary key, rearrange and save file

In [192]:
merged_df['genre_rel_id'] = range(1, len(dfrel) + 1)
merged_df = dfrel[['genre_rel_id', 'title', 'Genre']]
merged_df.to_csv('genre_rel.csv')